In [ ]:
!pip install accelerate evaluate rouge_score --quiet
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface_key")

from huggingface_hub import login
login(token=secret_value_0)

In [3]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch
import evaluate

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

2025-05-27 16:46:14.034544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748364374.223985      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748364374.281654      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [91]:
recipes = {
    "noodly tangle": {
        "file": "Classic Macaroni Salad Recipe with Video.jpg",
        "instruction": "Boil elbow macaroni in lightly salted water until tender (~8 min), then rinse under cold water and drain. Whisk together mayonnaise, sugar, vinegar, mustard, salt, and pepper, then toss with pasta. Stir in celery, onion, green pepper, carrot, and pimentos. Refrigerate at least 4 hours (or overnight) to meld flavors before serving."
    },
    "creamy pink layer": {
        "file": "Judys Strawberry Pretzel Salad Recipe.jpg",
        "instruction": "Preheat oven to 350°F. Mix crushed pretzels, melted butter, and sugar, press into a 9×13-inch pan, and bake 10 min. In a bowl, beat cream cheese with sugar, fold in whipped topping, spread over cooled crust, and chill 30 min. Dissolve Jell-O in boiling water, stir in frozen strawberries until thawed, pour over cream cheese layer, and refrigerate at least 1 hour before slicing."
    },
    "spiced shred stack": {
        "file": "Pulled Chicken Shawarma Sandwich Recipe.jpg",
        "instruction": "Combine shawarma spices in water with garlic and onion, simmer a whole chicken gently for 1 hour, turning every 20 min. Cool, shred meat, and reserve broth. Add bones back with water, simmer 1–2 hours, strain, and reduce broth by half. Return shredded chicken to reduced broth, cook until saucy, season with lemon juice and extra spices. Serve in bread or wraps."
    },
    "veggie garden layers": {
        "file": "Veggie Pizza Recipe.jpg",
        "instruction": "Preheat oven to 350°F and spray a 15×10-inch pan. Press crescent roll dough into pan, bake 10 min, and cool. Whip sour cream, cream cheese, ranch mix, dill, and garlic salt; spread over crust. Arrange broccoli, radish, onion, bell pepper, carrot, and celery on top; chill 1–2 hours, then cut into squares."
    },
    "charred sea pops": {
        "file": "Spicy Grilled Shrimp Recipe.jpg",
        "instruction": "Preheat grill to medium. Crush garlic and salt, stir in paprika, cayenne, olive oil, and lemon juice to form a paste. Toss shrimp with paste until coated. Oil grill grate lightly, then grill shrimp 2–3 min per side until opaque. Serve with lemon wedges."
    },
    "folded veggie bundle": {
        "file": "California Club Chicken Wraps Recipe.jpg",
        "instruction": "Whisk mayonnaise, yogurt, and chipotle chiles into a spread. Microwave tortillas 30 sec to warm. Spread chipotle mayo down the center, top with lettuce, cheese, avocado, bacon, onion, tomato, and chicken. Fold opposing edges over filling and roll into a tight wrap."
    },
    "layered olive loaf": {
        "file": "Real Nawlins Muffuletta Recipe.jpg",
        "instruction": "Chop olives, pepperoncini, cauliflower, cocktail onions, garlic, capers, celery, and carrot; mix with oregano, basil, pepper, celery seed, oils, and vinegar. Refrigerate covered 8 hours (or overnight). Halve Italian loaves, hollow centers, spread olive salad on both halves, layer salami, ham, mortadella, mozzarella, and provolone; reassemble and slice into quarters."
    },
    "poppy crunch tube": {
        "file": "Chicago-Style Hot Dog Recipe.jpg",
        "instruction": "Poach hot dog in simmering water 5 min, then steam poppy-seed bun 2 min. Place dog in bun and layer mustard, relish, onion, tomato wedges (one side), pickle spear (other side), sport peppers, and celery salt—no ketchup."
    },
    "spiced beef bun": {
        "file": "Tex-Mex Burger with Cajun Mayo Recipe.jpg",
        "instruction": "Preheat grill to medium-high and oil grate. Mix mayo with 1 tsp Cajun seasoning; set aside. Combine beef, onion, 3 tsp seasoning, jalapeño, garlic, and Worcestershire, form patties. Grill 5 min per side, topping with pepper Jack during last 2 min. Spread Cajun mayo on buns, add burger, lettuce, and tomato."
    },
    "cheesy wheel": {
        "file": "Campfire Pepperoni Pizza Recipe.jpg",
        "instruction": "Place pizza stone on grill over wood fire. Roll dough to thickness, cook one side 10 min, flip, spread sauce, cheese, and pepperoni. Tent with foil and cook until cheese melts (~10 min). Cool slightly, slice, and serve."
    }
}


image_dir = '/kaggle/input/recipie-high-res/recipes_images'
messages = []

messages.append({
    "role": "system",
    "content": [{
        "type": "text",
        "text": "You are a multimodal AI assistant specialized in cooking and recipe understanding. Your task is to generate concise cooking instructions (2–3 steps) from a food image and a noisy or vague dish title. Use your understanding of food preparation, ingredients, and cooking processes to infer what the dish likely is and how it is made, even when the input title is unclear or imprecise. Use natural, human-like language, and ensure that the instructions are practical and accurate for preparing the dish.You are also trained to mimic human-written summaries using few-shot examples provided. Do not invent exotic or unsafe steps. Prioritize clarity, brevity, and relevance in your response."
    }]
})

for vague_title, data in recipes.items():
    img = Image.open(f"{image_dir}/{data['file']}")
    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": f'Vague Title: "{vague_title}"\nInstruction:'}
        ]
    })
    messages.append({
        "role": "assistant",
        "content": [
            {"type": "text", "text": data['instruction']}
        ]
    })

# Example 1

In [106]:
test_image_path = f"{image_dir}/Dads Creamy Cucumber Salad Recipe.jpg"
test_vague_title = "chilled green slices"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [107]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Thinly slice cucumbers and place in a bowl. Whisk together yogurt, lemon juice, garlic, salt, and pepper. Pour dressing over cucumbers and gently toss to coat. Refrigerate for at least 30 minutes before serving.


# Example 2

In [111]:
test_image_path = f"{image_dir}/Bang Bang Chicken Casserole Recipe.jpg"
test_vague_title = "saucy rice bed"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [112]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Cook rice according to package directions. Shred chicken and toss with sriracha and honey. Layer rice in skillet, top with shredded chicken, melted mozzarella, and crispy bacon. Bake 20 min at 375°F until bubbly and golden. Garnish with green onions and sesame seeds.


# Example 3

In [114]:
test_image_path = f"{image_dir}/Strawberry Rhubarb Crumble Recipe.jpg"
test_vague_title = "tart berry crumble"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [115]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Combine berries, sugar, and cornstarch. Pour into a baking dish. Mix flour, butter, and sugar for crumble topping, sprinkle over berries. Bake at 375°F (190°C) for 30-40 minutes, or until topping is golden brown and berries are bubbling. Serve warm with vanilla ice cream.


# Live Example 1

In [86]:
test_image_path = f"{image_dir}/Chef Johns Nashville Hot Chicken Recipe.jpg"
test_vague_title = "fiery crunch pieces"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [ ]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

# Live Example 2

In [94]:
test_image_path = f"{image_dir}/Chicken Primavera Pasta Bake Recipe.jpg"
test_vague_title = "green swirl bake"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [ ]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)